In [1]:
import requests
import definition
import pandas as pd

In [2]:
session = requests.Session()

In [3]:
instrumentsDF = pd.read_pickle("data.pkl")

In [4]:
currency = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD', 'CAD']

In [10]:
def fetch_candles(pair_name, count, granularity):
    url = f"{definition.OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, headers=definition.SECURE_HEADER)
    return response.status_code, response.json()

In [14]:
def get_candles_df(json_response):
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']
    stuff = []
    for candle in json_response['candles']:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = candle['time']
        new_dict['volume'] = candle['volume']
        for price in prices:
            for oh in ohlc:
                new_dict[f"{price}_{oh}"] = candle[price][oh]
        stuff.append(new_dict)
    return pd.DataFrame.from_dict(stuff)

In [15]:
code , res = fetch_candles("EUR_USD", 10, "H1")

In [16]:
df = get_candles_df(res)

In [8]:
for p1 in currency:
    for p2 in currency:
        pair = f"{p1}_{p2}"